Here I will use different ensemble methods on several public models that perform successfully to see how they perform.  ALL CREDIT FOR EACH MODEL GOES TO THE ORIGINAL OWNER.

Models:

https://www.kaggle.com/nxrprime/train-infer-catalyst-pytorch-rnn-baseline
https://www.kaggle.com/xhlulu/openvaccine-simple-gru-model
https://www.kaggle.com/parmarsuraj99/neural-covid-vaccine
https://www.kaggle.com/tuckerarrants/openvaccine-gru-lstm

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import os

In [ ]:
submission = pd.read_csv('../input/stanford-covid-vaccine/sample_submission.csv')
csv1 = pd.read_csv('../input/train-infer-catalyst-pytorch-rnn-baseline/submission.csv')
csv2 = pd.read_csv('../input/neural-covid-vaccine/submission.csv')
csv3 = pd.read_csv('../input/openvaccine-gru-lstm/submission.csv')
csv4 = pd.read_csv('../input/openvaccine-simple-gru-model/submission.csv')

In [ ]:
files = [csv1, csv2, csv3, csv4]
total = pd.concat(files, axis=1)

In [ ]:
submission.head()

In [ ]:
total.head()

In [ ]:
col = list(submission.columns)[1:]
print(col)

## Mean

Score: 0.29375


In [ ]:
submission1 = submission
for c in col:
    submission1[c] = total[c].mean(axis=1)
submission1.head()

In [ ]:
submission1.to_csv('submission1.csv', index=False)

## Median

Score: 0.28293

In [ ]:
submission2 = submission
for c in col:
    submission2[c] = total[c].median(axis=1)
submission2.head()

In [ ]:
submission2.to_csv('submission2.csv', index=False)

## Mode

Score: 0.44178

In [ ]:
submission3 = submission
for c in col:
    submission3[c] = total[c].mode(axis=1)
submission3.head()

In [ ]:
submission3.to_csv('submission3.csv', index=False)

## MinMaxBestBaseStacking

Credit goes to @solomon with his notebook https://www.kaggle.com/solomonk/minmax-ensemble-0-9526-lb/, which I adapted to use here.  He wrote the original which can be found at https://github.com/QuantScientist/Deep-Learning-Boot-Camp/blob/master/Kaggle-PyTorch/PyTorch-Ensembler/utils.py .

Score: 0.27184

This one still needs to find the optimal cutoffs... more room to improve!

In [ ]:
submission4 = submission

cutoff_lo = 0.73
cutoff_hi = 0.33
    
for c in col:
    total[c+'_is_iceberg_max'] = total[c].max(axis=1)
    total[c+'_is_iceberg_min'] = total[c].min(axis=1)
    total[c+'_is_iceberg_mean'] = total[c].mean(axis=1)
    total[c+'_is_iceberg_median'] = total[c].median(axis=1)

    total[c+'_is_iceberg_base'] = csv3[c] #best score
    submission4[c] = np.where(np.all(total[c] > cutoff_lo, axis=1),
                                        total[c+'_is_iceberg_max'],
                                        np.where(np.all(total[c] < cutoff_hi, axis=1),
                                                 total[c+'_is_iceberg_min'],
                                                 total[c+'_is_iceberg_base']))
submission4.head()

In [ ]:
submission4.to_csv('submission4.csv', index=False)

## Blend (by percentage)

These percenages will be tuned more soon.

Score: 0.27049

In [ ]:
submission5 = submission
for c in col:
    submission5[c] = csv3[c] * 0.8 + csv4[c] * 0.1 + csv1[c] * 0.05 + csv2[c] *0.05
submission5.head()

In [ ]:
submission5.to_csv('submission5.csv', index=False)

## Summary

* Mean: 0.29375
* Median:  0.28293
* Mode:  0.44178
* MinMaxBestBaseStacking: 0.27184
* Blend: 0.27049

The blended one by percentage did the best.  The score for the best submissions I used was 0.27184, and the score with the blended one was 0.27049.

Interestingly, the score from the MinMaxBestBaseStacking was the same as the one for the GRU & LSTM model.

I will continue to update this kernel with improvements/different methods.

Please upvote/comment if you enjoyed!